In [0]:
#Imports
from pyspark.sql.functions import *

#configuration
spark.conf.set(
  "fs.azure.account.key.ecommercestorage5400.dfs.core.windows.net",
  "<password"
)

#dest path
gold_path = "abfss://ecommerce@ecommercestorage5400.dfs.core.windows.net/gold"
#src path
silver_path = "abfss://ecommerce@ecommercestorage5400.dfs.core.windows.net/silver"

#create from silver
df_silver = (
    spark.readStream
    .format('delta')
    .load(silver_path)
)

#aggregation: Total sales and total items sold per state per minute
df_gold = (
    df_silver
    .withWatermark('timestamp','1 minute')
    .groupBy(
        window('timestamp','1 minute'),
        'state')
    .agg(
        sum('total_amount').alias('total_sales'),
        sum('quantity').alias('total_items')
    )
    .select(
        col('window.start').alias('start'),
        col('window.end').alias('end'),
        'state',
        'total_sales',
        'total_items'
    )
)

(
    df_gold.writeStream
    .format('delta')
    .outputMode('append')
    .option('checkpointLocation', f'{gold_path}/_checkpoint')
    .start(gold_path)
)

In [0]:
df_gold =spark.readStream.format('delta').load(gold_path)
display(df_gold)

start,end,state,total_sales,total_items
